<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
!pip install spacy
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp312-cp312-win_amd64.whl (2.0 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)

   --- ------------------------------------  1/11 [typing-inspection]
   ---------- -----------------------------  3/11 [pydantic-core]
   ------------------ ---------------------  5/11 [annotated-types]
   ---------

In [3]:
# avoid decoding problems
df = pd.read_csv("train.csv")
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [5]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [39]:
!pip install --upgrade numpy h5py spacy


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.9 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.9 MB 2.1 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.9 MB 2.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.8/2.9 MB 1.2 MB/s eta 0:00:02
   -------------- ------------------------- 1.0/2.9 MB 1.1 MB/s eta 0:00:02
   ------------------ --------------------- 1.3/2.9 MB 1.0 MB/s eta 0:00:02
   ------------------ --------------------- 1.3/2.9 MB 1.0 MB/s eta 0:00:02
   --------------------- ------------------ 1.6/2.9 MB 964.5 kB/s eta 0:00:02
   --------------------- ------------------ 1.6/2.9 MB 964.5 kB/s eta 0:00:02
   ------------------------- -------------- 1.8/2.9 MB 907.1 kB/s eta 0:00:02
   ------------------------- -------------- 1.8/2.9 MB 907.1 kB/s eta 0:00:02
   -------------

In [41]:
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 2.6 MB/s eta 0:00:05
     ---- ----------------------------------- 1.6/12.8 MB 2.7 MB/s eta 0:00:05
     ------ --------------------------------- 2.1/12.8 MB 2.6 MB/s eta 0:00:05
     -------- ------------------------------- 2.6/12.8 MB 2.6 MB/s eta 0:00:04
     --------- ------------------------------ 3.1/12.8 MB 2.6 MB/s eta 0:00:04
     ----------- ---------------------------- 3.7/12.8 MB 2.6 MB/s eta 0:00:04
     ------------- -------------------------- 4.2/12.8 MB 2.6 MB/s eta 0:00:04
     --------------- ------------------------ 5.0/12.8 MB 2.7 MB/s eta 0:00:03
     ----------------- ---------------------- 5.5/12.8 MB 2.7 MB/s eta 0:00:03
     ------------------ --------------------- 5.8/12.8 MB 2.7 MB/s 

In [ ]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


 40%|████      | 163253/404290 [19:53<30:28, 131.82it/s] 

In [45]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████| 404290/404290 [45:18<00:00, 148.71it/s] 


In [11]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('preprocessed_data.csv'):
    dfnlp = pd.read_csv("preprocessed_data.csv",encoding='latin-1')
else:
    print("download preprocessed_data.csv from drive or run previous notebook")

if os.path.isfile('preprocessed_data(Basic).csv'):
    dfppro = pd.read_csv("preprocessed_data(Basic).csv",encoding='latin-1')
else:
    print("download preprocessed_data(Basic).csv from drive or run previous notebook")

In [13]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

AttributeError: 'DataFrame' object has no attribute 'q1_feats_m'

In [58]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,398782,1,75,76,13,13,12,26,0.46,0.874989,...,0.923070,1.0,1.0,0.0,13.0,0.855263,99,99,99,99
1,115086,0,48,56,13,16,8,24,0.33,0.666644,...,0.466664,1.0,1.0,3.0,13.5,0.224490,69,67,65,74
2,327711,0,104,119,28,21,4,38,0.11,0.000000,...,0.115384,0.0,0.0,6.0,23.0,0.047619,26,29,34,43
3,367788,0,58,145,14,32,1,34,0.03,0.000000,...,0.000000,0.0,0.0,17.0,21.5,0.050847,29,41,23,30
4,151235,0,34,49,5,9,3,13,0.23,0.749981,...,0.333330,1.0,0.0,4.0,7.0,0.542857,55,70,48,69


In [59]:
# data before preprocessing 
df2.head()

,id,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
0,398782,76,77.0,12,12,11,24,0.46
1,115086,49,57.0,12,15,7,23,0.30
2,327711,105,120.0,25,17,2,34,0.06
3,367788,59,146.0,12,30,0,32,0.00
4,151235,35,50.0,5,9,3,13,0.23


In [60]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-20.660634,-32.167675,2.436453,3.578740,-11.797639,-8.185505,63.807133,22.978929,2.836384,-3.285232,...,23.574625,-6.288059,-27.423591,21.784427,-29.386890,27.091443,28.375905,13.574432,-25.176610,40.798094
1,-29.945757,-55.347846,20.210477,43.394142,-3.518035,-22.161620,1.259152,23.305232,8.090744,-48.588327,...,4.270028,6.545918,-57.771808,6.523892,-2.134847,8.604939,20.239939,6.129380,-41.767028,38.066482
2,-26.433151,-7.260371,9.025158,13.887715,10.153314,-2.930556,47.487028,-9.282860,-7.071854,4.644652,...,23.804185,-2.787363,-22.982258,16.821755,-18.194621,0.158006,15.186030,7.984228,-16.467005,11.585927
3,24.912692,-21.611806,7.198035,-13.986901,-26.444345,-11.260169,-12.319716,-18.884277,-11.414455,14.632624,...,-24.020150,-9.783606,18.179139,44.030908,43.931766,-19.512516,26.911838,-7.789190,5.210079,12.931108
4,-23.205631,-69.044186,-9.514708,21.033791,-14.763523,-22.741932,49.175215,76.367652,4.505282,-25.713992,...,-0.240368,-21.056331,-67.267107,17.886652,-9.376377,28.389635,32.194397,45.022743,9.512707,66.023441


In [61]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-18.132199,-26.414625,6.357250,-2.776116,-10.028855,-2.629893,55.509493,17.499787,-4.939512,-1.176749,...,24.551529,-7.684039,-24.883157,6.526309,-24.260105,24.287874,23.207100,21.139578,-18.450893,36.307079
1,-32.206099,-68.997999,19.817884,29.907962,-28.345224,-30.291166,-5.844765,25.340969,12.290251,-36.099917,...,24.103431,-11.546866,-62.848033,15.441650,-9.592031,16.754833,26.424757,28.875602,-23.002500,16.389976
2,0.131700,-21.840135,22.282422,-0.836111,-11.782657,-5.972448,22.497115,-1.457767,-10.761685,-4.844840,...,39.676087,0.289840,-21.327893,31.244260,-15.868776,10.563364,-9.109160,19.774184,-11.013100,28.307958
3,-2.388362,-9.137740,2.312735,3.684138,-28.424859,-15.988031,4.731862,-10.062290,-12.901813,-12.599918,...,11.053748,-5.903925,0.677170,-7.418554,-5.314642,-12.190648,-3.195340,8.207225,6.415468,9.979167
4,-19.472628,-18.926319,10.867051,-5.542326,-19.215246,-11.929669,28.416746,6.577727,-0.498996,-1.649448,...,7.919799,-5.115303,-11.169791,-4.767877,-16.489244,13.219784,0.407731,17.408289,-16.857232,22.797968


In [62]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 24
Number of features in preprocessed dataframe : 8
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 224


In [73]:
print(df3_q1.shape)
print(df3_q2.shape)
print("Unique IDs in df3_q1:", df3_q1['id'].nunique())
print("Unique IDs in df3_q2:", df3_q2['id'].nunique())


(404290, 97)
(404290, 97)
Unique IDs in df3_q1: 90000
Unique IDs in df3_q2: 90000


In [ ]:
if not os.path.isfile('final_features.csv'):
    df3_q1['id'] = df1['id']
    df3_q2['id'] = df1['id']
    
    # Merge df1 and df2 with suffixes to avoid conflicts
    df1 = df1.merge(df2, on='id', how='left', suffixes=('_main', '_df2'))
    
    # Merge df3_q1 and df3_q2 with suffixes
    df2 = df3_q1.merge(df3_q2, on='id', how='left', suffixes=('_q1', '_q2'))
    
    # Merge everything into result
    result = df1.merge(df2, on='id', how='left')
    result.to_csv('final_features.csv', index=False)

In [75]:
# Merge Q1 and Q2 embeddings
X = df_q1_w2v.merge(df_q2_w2v, on='id', suffixes=('_q1','_q2'))

# Add labels
X['is_duplicate'] = df['is_duplicate']

# Separate features and target
y = X['is_duplicate']
X = X.drop(['id','is_duplicate'], axis=1)


NameError: name 'df_q1_w2v' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# Initialize Random Forest
rf = RandomForestClassifier(
    n_estimators=200,   # number of trees
    max_depth=10,       # max depth of each tree
    random_state=42,
    n_jobs=-1
)

# Train
rf.fit(X_train, y_train)

# Predict
y_pred_rf = rf.predict(X_test)

# Metrics
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest F1 Score:", f1_score(y_test, y_pred_rf))
print("Random Forest ROC-AUC:", roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))


In [ ]:
# Initialize XGBoost
xgb = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)

# Train
xgb.fit(X_train, y_train)

# Predict
y_pred_xgb = xgb.predict(X_test)

# Metrics
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("XGBoost F1 Score:", f1_score(y_test, y_pred_xgb))
print("XGBoost ROC-AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1]))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))
